In [ ]:
import os
import numpy as np
import pandas as pd
import healpy as hp
from astropy.time import Time, TimeDelta
from astropy.coordinates import SkyCoord
from astroplan import Observer
import astropy.units as u

from rubin_nights import connections
from rubin_nights import lfa_data
from rubin_nights import plot_utils as rn_plots

from IPython.display import display, HTML
import matplotlib.pyplot as plt

import pickle
from lsst.resources import ResourcePath
from rubin_scheduler.scheduler.schedulers import CoreScheduler
from rubin_scheduler.scheduler.features import Conditions
from rubin_scheduler.scheduler.model_observatory import ModelObservatory
from rubin_scheduler.utils import ddf_locations

import rubin_scheduler.scheduler.basis_functions as basis_functions
import rubin_scheduler.scheduler.detailers as detailers
from rubin_scheduler.skybrightness_pre import dark_m5
from rubin_scheduler.site_models import SeeingModel

import schedview.compute as schedview_compute

In [ ]:
def hp_laea(hp_array, alpha=None, label=None, vmin=None, vmax=None):
    hp.azeqview(hp_array, alpha=alpha, rot=(0, -90, 0), lamb=True, reso=17.5, min=vmin, max=vmax, title=label)
    hp.graticule()

def hp_moll(hp_array, alpha=None, label=None, vmin=None, vmax=None):
    hp.mollview(hp_array, alpha=alpha, min=vmin, max=vmax, title=label)
    hp.graticule()

In [ ]:
endpoints = connections.get_clients(tokenfile=os.path.join(os.path.expanduser("~"), ".lsst/usdf_rsp"), site="usdf")

In [ ]:
day_obs = Time(Time.now().mjd - 0.5, format='mjd', scale='tai').iso[0:10]
day_obs = "2025-09-18"

queue = 1


day_obs_time = Time(f"{day_obs}T12:00:00", format='isot', scale='tai')
tnow = Time.now()
observer = Observer.at_site('Rubin')
sunset = Time(observer.sun_set_time(day_obs_time, which='next', horizon=-0*u.deg), format='jd')
sunrise = Time(observer.sun_rise_time(day_obs_time, which='next', horizon=-0*u.deg), format='jd')
print(day_obs, 'sunset', sunset.iso,  'sunrise', sunrise.iso, 'now', Time.now().iso)

topic = "lsst.sal.Scheduler.logevent_target"
#topic = "lsst.sal.Scheduler.logevent_largeFileObjectAvailable"
targets = endpoints['efd'].select_time_series(topic, '*', sunset, sunrise, index=queue)
for c in ['ra', 'decl', 'skyAngle']:
    targets[c] = targets[c].astype(float)
if len(targets) == 0:
    display(endpoints['efd'].select_top_n(topic, '*', 2, index=queue))
print(len(targets))

topic = "lsst.sal.Scheduler.logevent_observation"
observations = endpoints['efd'].select_time_series(topic, '*', sunset, sunrise, index=queue)
for c in ['ra', 'decl', 'rotSkyPos']:
    observations[c] = observations[c].astype(float)
print(len(observations))

topic = "lsst.sal.Scheduler.logevent_largeFileObjectAvailable"
snapshots = endpoints['efd'].select_time_series(topic, '*', sunset, sunrise, index=queue)
print(len(snapshots))

if len(targets) > 0 and len(observations) > 0:
    to = pd.merge_asof(
                targets.sort_values("targetId").reset_index("time"),
                observations.sort_values("targetId").reset_index("time"),
                on="targetId",
                left_by=["ra", "decl", "skyAngle"],
                right_by=["ra", "decl", "rotSkyPos"],
                suffixes=("", "_o"),
                allow_exact_matches=True,
                direction="forward",
            )
    to.sort_values(by="time", inplace=True)
    to = to.astype({"targetId": int, "blockId": int, "skyAngle": float})
    print(len(to))
elif len(targets) > 0:
    to = pd.DataFrame(targets.sort_values("targetId").reset_index("time"))
    to['time_o'] = np.nan
    print("targets only")
else:
    to = None

#display(HTML(snapshots[['url']].to_html()))

In [ ]:
visits = endpoints['consdb'].get_visits("lsstcam", sunset, sunrise)

In [ ]:
visits.query("science_program == 'BLOCK-365'")[['target_name', 'observation_reason', 'science_program', 'band',  'zero_point_1s_pred', 'fwhm_eff', 'clouds']]

In [ ]:
# display(HTML(to.groupby('snapshotUri').first().reset_index()[['targetId', 'time', 'time_o',  'ra', 'decl', 'skyAngle', 
#                                                               'filter', 'note', 'targetName', 'airmass', 'snapshotUri', 'blockId' ]].to_html()))
to.groupby("snapshotUri").first()[['time', 'ra', 'decl', 'skyAngle', 'filter', 'note', 'targetName', 'airmass', 'targetId', 'blockId']]

In [ ]:
#display(HTML(targets[['ra', 'decl', 'skyAngle', 'filter', 'note', 'targetName', 'airmass', 'targetId', 'blockId', 'snapshotUri']].head().to_html()))

In [ ]:
uri = to.iloc[1].snapshotUri
uri = "https://s3.cp.lsst.org/rubinobs-lfa-cp/Scheduler:1/Scheduler:1/2025/09/18/Scheduler:1_Scheduler:1_2025-09-19T05:58:43.595.p"

sched, conditions = lfa_data.get_scheduler_snapshot(uri, at_usdf=True)
# Just check that these are the right kind of things 
assert isinstance(sched, CoreScheduler)
assert isinstance(conditions, Conditions)

In [ ]:
deg_sunset = -12
sunset = Time(observer.sun_set_time(day_obs_time, which='next', horizon=deg_sunset*u.deg), format='jd')
sunrise = Time(observer.sun_rise_time(day_obs_time, which='next', horizon=deg_sunset*u.deg), format='jd')
print("time now", Time.now().iso)
print("time conditions", Time(conditions.mjd, format='mjd', scale='utc').iso, "mjd", conditions.mjd)
print(f"sunset ({deg_sunset} deg)", sunset.iso)
print(f"sunrise ({deg_sunset} deg)", sunrise.iso)
print("conditions sun alt", np.degrees(conditions.sun_alt))

print("expected end of night", Time(conditions.sunrise - 3/24, format='mjd', scale='tai').iso)
print("now - conditions (hours)", (Time.now() - Time(conditions.mjd, format='mjd', scale='utc')).jd * 24, conditions.mjd)

In [ ]:
len(conditions.cloud_maps.mjds), conditions.mounted_bands, conditions.current_band, 

In [ ]:
conditions.cloud_maps.mjds, conditions.cloud_maps.cloud_extinction_hparrays
cmaps = conditions.cloud_maps.cloud_extinction_hparrays[-1]
hp_moll(cmaps)
np.nanmax(cmaps)

In [ ]:
if conditions.targets_of_opportunity is not None:
    n_toos = len(conditions.targets_of_opportunity)
    print("n_toos", n_toos)

    too_idx = -1
    print(conditions.targets_of_opportunity[too_idx].__dict__)
    print((conditions.mjd - conditions.targets_of_opportunity[too_idx].mjd_start)*24*60)
    print(len(np.where(conditions.targets_of_opportunity[too_idx].footprint == True)[0]))
    
    hpid = np.where(conditions.targets_of_opportunity[too_idx].footprint == True)
    print(hpid)
    print('too healpix positions', hp.pix2ang(nside=32, ipix=hpid, lonlat=True))
    print('center of too', np.degrees([conditions.targets_of_opportunity[0].ra_rad_center, conditions.targets_of_opportunity[0].dec_rad_center]))

In [ ]:
# DDF basis functions?
# ddfs = ddf_locations(skycoords=True)
# s = sched.survey_lists[0][0]
# reward = np.zeros(hp.nside2npix(32))
# for bf, weight in zip(s.basis_functions, s.basis_weights):
#         basis_value = bf(conditions)
#         reward += basis_value * weight
# hp_moll(reward)
# for ddf in ddfs:
#     hp.projplot(ddfs[ddf].ra.deg, ddfs[ddf].dec.deg, 'r.', lonlat=True,)

In [ ]:
# DDF check
# s = sched.survey_lists[0][0]
# s.calc_reward_function(conditions)
# so = pd.DataFrame(s.obs_wanted)
# so.query("mjd <= @conditions.mjd and flush_by_mjd >= @conditions.mjd")
# soo = pd.DataFrame(s._check_list(conditions))
# print(len(soo))
# s.generate_observations_rough(conditions)

In [ ]:
# DDF obs wanted?
# s = sched.survey_lists[0][0]
# print(len(s.obs_wanted))
# print(len(s.obs_wanted), 'start', s.obs_wanted[0]['mjd'], 'flush', s.obs_wanted[0]['flush_by_mjd'], conditions.mjd, conditions.mjd - s.obs_wanted[0]['mjd'])
# print(len(s.obs_wanted), 'start', s.obs_wanted[0]['mjd'], 'flush', s.obs_wanted[0]['flush_by_mjd'], conditions.mjd, conditions.mjd - s.obs_wanted[0]['mjd'])
# s.generate_observations_rough(conditions)
# s.obs_wanted['observed']

In [ ]:
# TOO stuff 
# ## sched.update_conditions(conditions)
# s = sched.survey_lists[0][0]
# #s.generate_observations(conditions)
# sched.request_observation(whole_queue=True)
# q = pd.DataFrame(s.obs_wanted)[['RA', 'dec', 'mjd', 'flush_by_mjd', 'band', 'scheduler_note', 'alt_min', 'alt_max', 'observed']]

# too plot
# fig, ax = plt.subplots(figsize=(8, 8))
# ax.set_aspect('equal', 'box')
# theta = np.arange(0, 2*np.pi, 0.05)
# for i, row in q.iterrows():
#     x = 1.75*(np.cos(theta) + np.sin(theta)) + np.degrees(row.RA)
#     y = 1.75*(np.cos(theta) - np.sin(theta)) + np.degrees(row.dec)
#     plt.plot(x, y, color='gray', marker='.', linestyle='', markersize=2, alpha=0.5)
# #plt.plot(np.degrees(q.RA), np.degrees(q.dec), 'b+')
# too = conditions.targets_of_opportunity[-1]
# allhpcen = hp.pix2ang(nside=32, ipix=np.arange(0, hp.nside2npix(32)), lonlat=True)
# window = np.where((allhpcen[0] > 226) & (allhpcen[0] < 242) & (allhpcen[1]> -44) & (allhpcen[1]<-30))[0]
# plt.plot(allhpcen[0][window], allhpcen[1][window], 'ko')
# hpcen = hp.pix2ang(nside=32, ipix=np.where(too.footprint == True)[0], lonlat=True)
# plt.plot(hpcen[0], hpcen[1], 'ro')
# q

In [ ]:
print(Time(conditions.mjd, format='mjd', scale='utc').iso)
#print(sched.survey_lists)
rewards = schedview_compute.make_scheduler_summary_df(sched, conditions)
rewards[['list_index', 'survey_index', 'survey_name_with_id', 'reward']]

In [ ]:
# Rewards in a tier 
tier = 4
for idx, s in enumerate(sched.survey_lists[tier]):
    if "Long Gap" in s.survey_name:
        si = s.blob_survey
        rew = si.calc_reward_function(conditions)
        if isinstance(rew, (float, int)):
            print(f"{tier} {idx} {si.survey_name} {rew:.3f}")
        else:
            print(f"{tier} {idx} {si.survey_name} {np.nanmax(rew):.3f}")
            fig = hp_laea(rew, label=si.survey_name)
    rew = s.calc_reward_function(conditions)
    if isinstance(rew, (float, int)):
        print(f"{tier} {idx} {s.survey_name} {rew:.3f}")
    else:
        print(f"{tier} {idx} {s.survey_name} {np.nanmax(rew):.3f}")
        fig = hp_laea(rew, label=s.survey_name)

In [ ]:
# Rewards for a survey (or two)
tier = 5
idx = 0
s = sched.survey_lists[tier][idx]#.blob_survey
s2 = sched.survey_lists[tier][idx]#.blob_survey
if (s == s2) | (s2 is None):
    print(s.survey_name, np.nanmax(s.calc_reward_function(conditions)))
    for i, bf in enumerate(s.basis_functions):
        feas = bf.check_feasibility(conditions)
        val = bf(conditions)
        if isinstance(val, (float, int)):        
            print(i, feas, bf.label(), f"{val:.3f}")
        else:
            if s.roi_hpid is not None:
                print(i, feas, bf.label(), f"{val[s.roi_hpid]:.3f}")
            else:
                print(i, feas, bf.label(), f"{np.nanmax(val):.3f}")
                #hp_laea(val, label=bf.label())
else:
    # Assume that same tier = basically the same survey .. 
    # (that would be when you might do this anyway I think)
    print(s.survey_name, s2.survey_name)
    for i, (bf, bf2) in enumerate(zip(s.basis_functions, s2.basis_functions)):
        feas = bf.check_feasibility(conditions)
        feas2 = bf2.check_feasibility(conditions)
        val = bf(conditions)
        val2 = bf2(conditions)
        if isinstance(val, (float, int)):        
            print(i, feas, feas2, bf.label(), bf2.label(), f"{val:.3f}", f"{val2:.3f}")
        else:
            if (s.roi_hpid is not None) | (s2.roi_hpid is not None):
                if s.roi_hpid is not None:
                    val = val[s.roi_hpid]
                if s2.roi_hpid is not None:
                    val2 = val2[s2.roi_hpid]
                print(i, feas, feas2, bf.label(), bf2.label(), f"{val:.3f}", f"{val2:.3f}")
            else:
                print(i, feas, feas2, bf.label(), bf2.label(), f"{np.nanmax(val):.3f}", f"{np.nanmax(val2):.3f}")
                #hp_laea(val, label=bf.label())

In [ ]:
# Combined footprint/masks for above survey

footprintbf = [bf for bf in s.basis_functions if 'Footprint' in bf.label()]
maskbf = [bf for bf in s.basis_functions if 'Avoid' in bf.label() or 'Shadow' in bf.label() or "HaMask" in bf.label()]
mask = np.zeros(hp.nside2npix(32))
for mbf in maskbf:
    mask += mbf(conditions)
mask = np.where(np.isnan(mask), 0.3, 1)
fp = np.zeros(hp.nside2npix(32))
for fbf in footprintbf:
    fp += fbf(conditions)
hp.mollview(fp, alpha=mask, rot=(330, 0, 0))
hp.graticule()
#hp_laea(fp, alpha=mask)

In [ ]:
survey_info = {'nside': 32, 'survey_start': Time("2025-06-20T12:00:00", scale='tai'), "downtimes": "ideal"}

def setup_observatory_summit(
    survey_info: dict, seeing: float | None = None, clouds: bool = False
) -> ModelObservatory:
    """Configure a `summit-10` model observatory.
    This approximates average summit performance at present.

    Parameters
    ----------
    survey_info
        The survey_info dictionary returned by `survey_times`.
        Note that the survey_info carries the downtime information,
        as well as the survey_start information.
    seeing
        If specified (as a float), then the constant seeing model will be
        used, delivering atmospheric seeing with `seeing` arcsecond values.
    clouds
        If True, use our standard cloud downtime model.
        If False, use the 'ideal' cloud model resulting in no downtime.

    Returns
    -------
    model_observatory
        ModelObservatory complete with seeing model, cloud (weather downtime)
        model, and also downtimes (due to engineering).
    """
    if seeing is not None:
        seeing_data = ConstantSeeingData()
        seeing_data.fwhm_500 = seeing
    else:
        # Else use standard seeing data
        seeing_data = None
    # Use a bigger system contribution
    seeing_model = SeeingModel(telescope_seeing=0.52)

    if clouds:
        cloud_data = None
    else:
        cloud_data = "ideal"

    observatory = ModelObservatory(
        nside=survey_info["nside"],
        mjd=survey_info["survey_start"].mjd,
        mjd_start=survey_info["survey_start"].mjd,
        cloud_data=cloud_data,
        seeing_data=seeing_data,
        wind_data=None,
        downtimes=survey_info["downtimes"],
        sim_to_o=None,
    )
    observatory.seeing_model = seeing_model
    # "10 percent TMA" - but this is a label from the summit, not 10% in all
    observatory.setup_telescope(
        azimuth_maxspeed=1.0,
        azimuth_accel=1.0,
        azimuth_jerk=4.0,
        altitude_maxspeed=4.0,
        altitude_accel=1.0,
        altitude_jerk=4.0,
        settle_time=3.45,  # more like current settle average
    )
    observatory.setup_camera(band_changetime=130, readtime=3.07)

    return observatory

observatory = setup_observatory_summit(survey_info, seeing=None, clouds=False) 

In [ ]:
sched.update_conditions(conditions)
obs = sched.request_observation(whole_queue=True)

In [ ]:
for o in obs:
    obs = observatory.observe(o)
    sched.add_observation(obs)